### Оглавление

1. [Подготовка данных](#Подготовка-данных)
2. [Пропуски в данных](#Пропуски-в-данных)
3. [Кодирование категориальных признаков](#Кодирование-категориальных-признаков)
4. [Кодирование даты и времени](#Кодирование-даты-и-времени)

# Подготовка данных

Предлагается рассмотреть корпус [Bank Marketing](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing).

Корпус содержит результаты обзвонов клиентов банка. Требуется предсказать открыл ли клиент вклад.

### Загрузка корпуса

Корпус автоматически скачается. Обратите внимание на аргументы `pd.read_csv()`.

In [1]:
import os
import tempfile
import urllib
import zipfile
from contextlib import contextmanager

import numpy as np
import pandas as pd


DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
TARGET = "y"

@contextmanager
def download(url):
    with tempfile.TemporaryDirectory() as root:
        os.path.basename(url)
        filename = os.path.basename(urllib.parse.urlparse(url).path)
        path = os.path.join(root, filename)
        urllib.request.urlretrieve(url, path)
        yield path


@contextmanager
def unzip(filename):
    with tempfile.TemporaryDirectory() as root:
        with zipfile.ZipFile(filename, "r") as zfp:
            zfp.extractall(root)
        yield root

        
def parse_names(filename):
    names = []
    with open(filename) as fp:
        for line in fp:
            if not line.startswith("@attribute"):
                continue
            names.append(line.split()[1])
    return names


with download(DATA_URL) as data_path:
    with unzip(data_path) as data_root:
        data = pd.read_csv(os.path.join(data_root, "bank-full.csv"),
                           sep=";",
                           na_values=["unknown"])  # Пропуски в .csv файле помечены словом "unknown".
        
        
# Усложним задачу: добавим пропуски в числовой признак.
np.random.seed(0)
mask = np.random.rand(len(data)) < 0.7
data["balance"].where(mask, other=np.NaN, inplace=True)

### Exploratory data analysis (EDA)

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45211 non-null  int64  
 1   job        44923 non-null  object 
 2   marital    45211 non-null  object 
 3   education  43354 non-null  object 
 4   default    45211 non-null  object 
 5   balance    31751 non-null  float64
 6   housing    45211 non-null  object 
 7   loan       45211 non-null  object 
 8   contact    32191 non-null  object 
 9   day        45211 non-null  int64  
 10  month      45211 non-null  object 
 11  duration   45211 non-null  int64  
 12  campaign   45211 non-null  int64  
 13  pdays      45211 non-null  int64  
 14  previous   45211 non-null  int64  
 15  poutcome   8252 non-null   object 
 16  y          45211 non-null  object 
dtypes: float64(1), int64(6), object(10)
memory usage: 5.9+ MB


In [3]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,NaN,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506.0,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1.0,no,no,NaN,5,may,198,1,-1,0,NaN,no


Каждая из 45211 строк таблицы соотвествует одному звонку. В корпусе 16 признаков. Некоторые могут быть пропущены. Нужно предсказать поведение клиента (значение поля `y`).

### Train / test split

Выделим тренировочную и тестовую части корпуса. Также отделим признаки от целевого значения.

In [4]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.2, random_state=0)

X_train = data_train.drop([TARGET], axis=1)
X_test = data_test.drop([TARGET], axis=1)
y_train = data_train[TARGET]
y_test = data_test[TARGET]

print("Train size", len(X_train))
print("Test size", len(X_test))

Train size 36168
Test size 9043


# Пропуски в данных

Как мы увидели, часто значение признаков равно `NaN`. Это значит, что оно пропущено в исходном корпусе.

In [5]:
data.isna().any().sum()  # Сколько признаков содержат неизвестные значения.

5

In [6]:
# Увеличим число отображаемых строк.
try:
    pd.set_option("display.height", 30)  # Старые версии pandas.
except Exception:
    pd.set_option("display.max_rows", 30)  # Новые версии.
nan_count = data.isna().sum()  # Сколько неизвестных значений содержит каждый признак.
nan_count[nan_count > 0]

job            288
education     1857
balance      13460
contact      13020
poutcome     36959
dtype: int64

Большинство ML моделей не поддерживают пропуски.

**Задание 1.** Предлагается заполнить пропуски в числовых полях `X_train` и `X_test` средним значением из `X_train`.

*Подсказка*. Можно воспользоваться функциями `<df>.mean()` и `<df\>.fillna()`. Обратите внимание на аргумент `numeric_only` функции `mean`.

In [7]:
#
# Ваш код.
#

means = X_train.mean(numeric_only=True)
X_train_new = X_train.fillna(means)
X_test_new = X_test.fillna(means)

# Проверки.

assert abs(X_train_new.loc[9894, "balance"] - 1362.268770) < 1e-6
assert abs(X_test_new.loc[13318, "balance"] - 1362.268770) < 1e-6
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

Check OK!


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
13932,57,admin.,divorced,secondary,no,658.00000,no,no,cellular,10,jul,724,1,-1,0,NaN
9894,37,NaN,married,NaN,no,1362.26877,no,no,NaN,9,jun,63,1,-1,0,NaN
39946,35,technician,divorced,secondary,no,2823.00000,yes,no,cellular,2,jun,102,4,96,2,failure
9217,35,admin.,married,secondary,no,1362.26877,yes,yes,NaN,5,jun,247,1,-1,0,NaN
4124,38,services,single,tertiary,no,1362.26877,yes,no,NaN,19,may,138,1,-1,0,NaN


**Задание 2.** В корпусе остались пропущенные признаки. Pandas не смог вычислить среднее для категориальных признаков. Заполните их наиболее вероятным значением.

*Подсказка*. Можно воспользоваться функциями `<df>.mode()` и `<df\>.fillna()`. Функция `mode` возвращает таблицу с несколькими строками на случай, если мода не одна. Нужно взять первую строку.

In [8]:
#
# Ваш код.
#

values = X_train.mode().iloc[0]
X_train_new = X_train.fillna(values)
X_test_new = X_test.fillna(values)

# Проверки.

assert X_train_new.loc[9894, "job"] == "blue-collar"
assert X_test_new.loc[18883, "job"] == "blue-collar"
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

Check OK!


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
13932,57,admin.,divorced,secondary,no,658.00000,no,no,cellular,10,jul,724,1,-1,0,failure
9894,37,blue-collar,married,secondary,no,1362.26877,no,no,cellular,9,jun,63,1,-1,0,failure
39946,35,technician,divorced,secondary,no,2823.00000,yes,no,cellular,2,jun,102,4,96,2,failure
9217,35,admin.,married,secondary,no,1362.26877,yes,yes,cellular,5,jun,247,1,-1,0,failure
4124,38,services,single,tertiary,no,1362.26877,yes,no,cellular,19,may,138,1,-1,0,failure


Проверим, что пропусков не осталось.

In [9]:
assert X_train.isna().any().sum() == 0
assert X_test.isna().any().sum() == 0

# Кодирование категориальных признаков

###  Как понять, что признак категориальный?

Идея - категориальные признаки те, в которых значения - строки или число уникальных значений достаточно мало.

In [10]:
def find_cat(data, num_uniq=10):
    columns = []
    for name in data.columns:
        message = name
        if type(data[name][0]) == str:
            message += " строка,"
        if data[name].nunique() <= num_uniq:
            message += " мало уникальных"
        if message != name:
            columns.append(name)
            print(message)
    return columns
            
find_cat(data)

job строка,
marital строка, мало уникальных
education строка, мало уникальных
default строка, мало уникальных
housing строка, мало уникальных
loan строка, мало уникальных
contact мало уникальных
month строка,
poutcome мало уникальных
y строка, мало уникальных


['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome',
 'y']

**Задание 3.** Реализуйте one-hot кодирование категориальных признаков, перечисленных в `CAT_COLUMNS`.

*Подсказка*. Следите, чтобы train и test кодировались одинаково. Можно воспользоваться функциями `pd.cancat()` и `pd.get_dummies(<df>)`.

In [11]:
CAT_COLUMNS = ["job", "marital", "education", "contact", "poutcome"]

#
# Ваш код.
#

joined = pd.get_dummies(pd.concat([X_train, X_test], axis=0), columns=CAT_COLUMNS)
X_train_new = joined[:len(X_train)]
X_test_new = joined[len(X_train):]

# Проверки.

assert X_train_new.loc[13932, "job_admin."] == 1
assert X_test_new.loc[14001, "education_tertiary"] == 1
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

Check OK!


,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_primary,education_secondary,education_tertiary,contact_cellular,contact_telephone,poutcome_failure,poutcome_other,poutcome_success
13932,57,no,658.00000,no,no,10,jul,724,1,-1,...,0,0,0,1,0,1,0,1,0,0
9894,37,no,1362.26877,no,no,9,jun,63,1,-1,...,1,0,0,1,0,1,0,1,0,0
39946,35,no,2823.00000,yes,no,2,jun,102,4,96,...,0,0,0,1,0,1,0,1,0,0
9217,35,no,1362.26877,yes,yes,5,jun,247,1,-1,...,1,0,0,1,0,1,0,1,0,0
4124,38,no,1362.26877,yes,no,19,may,138,1,-1,...,0,1,0,0,1,1,0,1,0,0


Покажем, как можно сгруппировать числовые признаков по категориальным.

Например, для каждого значения признака `education` из нашей таблички можно посчитать среднее значение `balance`.

In [12]:
def group_mean(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

group_mean(data, "education", "balance")

0        1745.256643
1        1151.918662
2        1151.918662
3                NaN
4                NaN
            ...     
45206    1745.256643
45207    1203.009834
45208    1151.918662
45209    1151.918662
45210    1151.918662
Name: education, Length: 45211, dtype: float64

**Задание 4.** Замените `yes` и `no` в столбцах `YN_COLUMNS` на 1 и 0 соотвественно.

*Подсказка.* Можно воспользоваться методами `<df>[<column>].map()` и `<df>.drop([<column>], axis=1)`.

In [13]:
YN_COLUMNS = ["default", "housing", "loan"]
MAPPING = {"yes": 1, "no": 0}

#
# Ваш код.
#

X_train_new = X_train.copy()
X_test_new = X_test.copy()

for column in YN_COLUMNS:
    X_train_new[column] = X_train[column].map(MAPPING)
    X_test_new[column] = X_test[column].map(MAPPING)
    
# Проверки.

assert X_train_new.loc[39946, "housing"] == 1
assert X_test_new.loc[32046, "default"] == 0
print("Check OK!")

X_train = X_train_new
X_test = X_test_new
X_train.head()

Check OK!


,age,default,balance,housing,loan,day,month,duration,campaign,pdays,...,marital_married,marital_single,education_primary,education_secondary,education_tertiary,contact_cellular,contact_telephone,poutcome_failure,poutcome_other,poutcome_success
13932,57,0,658.00000,0,0,10,jul,724,1,-1,...,0,0,0,1,0,1,0,1,0,0
9894,37,0,1362.26877,0,0,9,jun,63,1,-1,...,1,0,0,1,0,1,0,1,0,0
39946,35,0,2823.00000,1,0,2,jun,102,4,96,...,0,0,0,1,0,1,0,1,0,0
9217,35,0,1362.26877,1,1,5,jun,247,1,-1,...,1,0,0,1,0,1,0,1,0,0
4124,38,0,1362.26877,1,0,19,may,138,1,-1,...,0,1,0,0,1,1,0,1,0,0


# Кодирование даты и времени

**Задание 5.** Реализуйте периодическое кодирование месяца (в новые поля `month_cos` и `month_sin`):

$cos(2 * \pi * \frac{x}{period})$, $sin(2 * \pi * \frac{x}{period})$

*Подсказка.* Можно воспользоваться методами `<df>[<column>].map()` и `<df>.drop([<column>], axis=1)`.

In [14]:
ORDER = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
MONTH_MAPPING = {month: i for i, month in enumerate(ORDER)}

#
# Ваш код.
#

# X_train_new = ...
# X_test_new = ...

X_train_new = X_train.drop(["month"], axis=1)
order = X_train["month"].map(MONTH_MAPPING)
X_train_new["month_cos"] = order.map(lambda x: np.cos(2 * np.pi * x / 12))
X_train_new["month_sin"] = order.map(lambda x: np.sin(2 * np.pi * x / 12))

X_test_new = X_test.drop(["month"], axis=1)
order = X_test["month"].map(MONTH_MAPPING)
X_test_new["month_cos"] = order.map(lambda x: np.cos(2 * np.pi * x / 12))
X_test_new["month_sin"] = order.map(lambda x: np.sin(2 * np.pi * x / 12))

# Проверки.

assert abs(X_train_new.loc[13932, "month_cos"] + 1) < 1e-6
assert abs(X_test_new.loc[32046, "month_sin"] - 1) < 1e-6
print("Check OK!")

X_train = X_train_new
X_test = X_test_new
X_train.head()

Check OK!


,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,education_primary,education_secondary,education_tertiary,contact_cellular,contact_telephone,poutcome_failure,poutcome_other,poutcome_success,month_cos,month_sin
13932,57,0,658.00000,0,0,10,724,1,-1,0,...,0,1,0,1,0,1,0,0,-1.000000,1.224647e-16
9894,37,0,1362.26877,0,0,9,63,1,-1,0,...,0,1,0,1,0,1,0,0,-0.866025,5.000000e-01
39946,35,0,2823.00000,1,0,2,102,4,96,2,...,0,1,0,1,0,1,0,0,-0.866025,5.000000e-01
9217,35,0,1362.26877,1,1,5,247,1,-1,0,...,0,1,0,1,0,1,0,0,-0.866025,5.000000e-01
4124,38,0,1362.26877,1,0,19,138,1,-1,0,...,0,0,1,1,0,1,0,0,-0.500000,8.660254e-01


Выходные значения тоже нужно преобразовать в 0,1

In [16]:
y_train = y_train.map(MAPPING)
y_test = y_test.map(MAPPING)
y_train.head()

13932    1
9894     0
39946    0
9217     0
4124     0
Name: y, dtype: int64

# Проверим, что теперь все данные числовые

In [17]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36168 entries, 13932 to 2732
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  36168 non-null  int64  
 1   balance              36168 non-null  float64
 2   day                  36168 non-null  int64  
 3   duration             36168 non-null  int64  
 4   campaign             36168 non-null  int64  
 5   pdays                36168 non-null  int64  
 6   previous             36168 non-null  int64  
 7   job_admin.           36168 non-null  uint8  
 8   job_blue-collar      36168 non-null  uint8  
 9   job_entrepreneur     36168 non-null  uint8  
 10  job_housemaid        36168 non-null  uint8  
 11  job_management       36168 non-null  uint8  
 12  job_retired          36168 non-null  uint8  
 13  job_self-employed    36168 non-null  uint8  
 14  job_services         36168 non-null  uint8  
 15  job_student          36168 non-nu

In [14]:
# Dump.
#X_train.to_csv("data/x-train.csv")
#X_test.to_csv("data/x-test.csv")
#y_train.to_csv("data/y-train.csv")
#y_test.to_csv("data/y-test.csv")